In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier 
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, KFold, LeaveOneOut
from sklearn.preprocessing import Imputer
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from random import randint
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import category_encoders as ce
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from catboost import CatBoostClassifier

In [2]:
train_A = pd.read_csv('data/A_hhold_train.csv', index_col='id')
train_B = pd.read_csv('data/B_hhold_train.csv', index_col='id')
train_C = pd.read_csv('data/C_hhold_train.csv', index_col='id')

In [3]:
test_A = pd.read_csv('data/A_hhold_test.csv', index_col='id')
test_B = pd.read_csv('data/B_hhold_test.csv', index_col='id')
test_C = pd.read_csv('data/C_hhold_test.csv', index_col='id')

In [4]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]

In [5]:
#funcetion used to centrailize the numeric data 
def standardize(df, numeric_only=True): 
    
    """
    :type df: pd.DataFrame
    :type numeric_only: bool
    
    :rtype df: pd.DataFrame
    """
    
    numeric = df.select_dtypes(include=['int64', 'float64'])
    
    # subtracy mean and divide by std
    df[numeric.columns] = (numeric - numeric.mean()) / numeric.std()
    
    return df

In [6]:
#function used to preprocess the data(simply drop the poor and country columns)
def preporcess(original_train, original_test, op='one_hot'):
    
    """
    :type original_train: pd.DataFrame
    :type original_test: pd.DataFrame
    :type op: string
    
    :rtype x_train: pd.DataFrame
    :rtype y_train: pd.DataFrame
    :rtype x_test: pd.DataFrame
    """
    
    #Get the copy of orginal data
    x_train = original_train.copy()
    x_test = original_test.copy()

    #Get the label of the train set
    y_train = pd.DataFrame(x_train['poor'])
    
    #drop the 'poor' and 'country' columns
    x_train = x_train.drop(['poor', 'country'], axis=1)
    x_test = x_test.drop(['country'], axis=1)
    
    #get the list of coulumns that need to be encoded
    columns = list(x_train.select_dtypes(include=['object']).columns)
    
    #encode the features according to the 'op' operations
    if op == 'one_hot':
        encoder = ce.one_hot.OneHotEncoder(cols=columns)
    elif op == 'binary':
        encoder = ce.binary.BinaryEncoder(cols=columns)
    elif op == 'ordinal':
        encoder = ce.ordinal.OrdinalEncoder(cols=columns)
    elif op == 'backward':
        encoder = ce.backward_difference.BackwardDifferenceEncoder(cols=columns)
    elif op == 'polynomial':
        encoder = ce.polynomial.PolynomialEncoder(cols=columns)
    elif op == 'sum':
        encoder = ce.sum_coding.SumEncoder(cols=columns)
    elif op == 'hashing':
        encoder = ce.hashing.HashingEncoder(cols=columns)
    elif op == 'helmert':
        encoder = ce.helmert.HelmertEncoder(cols=columns)
    
    #fit the encoder and transform the datasets
    encoder.fit(x_train)
    x_train = encoder.transform(x_train)
    x_test = encoder.transform(x_test)
    
    #for columns with Nan, plug in with median value
    imputed_train = x_train.copy()
    imputed_test = x_test.copy()
    
    # make new columns indicating what will be imputed
    cols_with_missing = (col for col in x_train.columns 
                                 if x_train[col].isnull().any())
    
    #insert new colums and tell which row is missing 
    for col in cols_with_missing:
        imputed_train[col + '_was_missing'] = imputed_train[col].isnull()
        imputed_test[col + '_was_missing'] = imputed_test[col].isnull()
    
    imp = Imputer(missing_values='NaN', strategy='median',axis=0)
    x_train = imp.fit_transform(imputed_train)
    x_test = imp.transform(imputed_test)
       
    return x_train, y_train.values.ravel(), x_test

In [7]:
x_train_A, y_train_A, x_test_A = preporcess(original_train=train_A, original_test=test_A, op='one_hot')
x_train_B, y_train_B, x_test_B = preporcess(original_train=train_B, original_test=test_B, op='one_hot')
x_train_C, y_train_C, x_test_C = preporcess(original_train=train_C, original_test=test_C, op='one_hot')

In [8]:
def cross_validate(model, X, y, cv=5, oversample=False, early_stop=False):
    
    seed = randint(1,9999)
    result_log_loss = np.zeros(cv)
    result_roc_auc = np.zeros(cv)
    
    kf = StratifiedKFold(n_splits=cv)
    
    i = 0
    
    for train_index, dev_index in kf.split(X,y):
        x_train = X[train_index]
        y_train = y[train_index]
        x_dev = X[dev_index]
        y_dev = y[dev_index]
        
        if oversample == True:
            x_train, y_train = RandomOverSampler().fit_sample(x_train, y_train)
            
        if early_stop == True:
            model.fit(X=x_train, y=y_train, eval_set=[(x_train,y_train),(x_dev,y_dev)], eval_metric='logloss', early_stopping_rounds=20)
        else:
            model.fit(x_train, y_train)
        
        pred_proba = model.predict_proba(x_dev)[:,-1]
        
        result_log_loss[i] = log_loss(y_pred=pred_proba, y_true=y_dev)
        result_roc_auc[i] = roc_auc_score(y_score=pred_proba, y_true=y_dev)
        
        i += 1
        
    return result_log_loss.mean(), result_roc_auc.mean()

In [9]:
def gridSearch(model, grid_params, X, y, oversample=False, early_stop=False, cv=5):
    
    grids = ParameterGrid(grid_params)
    result_score = np.zeros([len(grids), 2])
    
    i = 0
    
    for grid in grids:
        model.set_params(**grid)
        logScore, aucScore = cross_validate(model, X, y, cv, oversample, early_stop)
        result_score[i][0] = logScore
        result_score[i][1] = aucScore
        i += 1
    
    return grids, result_score

In [10]:
def model_feature_evaluate(feature_model, predict_model, X, y, oversample=False, early_stop=False, cv=5):
    feature_model.fit(X,y)
    feature_model = SelectFromModel(feature_model, prefit=True)
    X_new = feature_model.transform(X)
    logScore, aucScore = cross_validate(predict_model, X_new, y, cv, oversample, early_stop)
    return logScore, aucScore

def base_model_evaluate(model, X, y, oversample=False, early_stop=False, cv=5):
    logScore, aucScore = cross_validate(model, X, y, cv, oversample, early_stop)
    return logScore, aucScore

def feature_transform(feature_model, X_train, Y_train, X_test):
    feature_model.fit(X_train, Y_train)
    model = SelectFromModel(feature_model, prefit=True)
    X_train_new = model.transform(X_train)
    X_test_new = model.transform(X_test)
    return X_train_new, X_test_new

In [11]:
x_train_new_A, x_test_new_A = feature_transform(XGBClassifier(), x_train_A, y_train_A, x_test_A)
x_train_new_B, x_test_new_B = feature_transform(XGBClassifier(), x_train_B, y_train_B, x_test_B)
x_train_new_C, x_test_new_C = feature_transform(XGBClassifier(), x_train_C, y_train_C, x_test_C)

# Country A Model Parameters

In [19]:
#xgb data
#xgb
xgb_params_A = {}
xgb_params_A['learning_rate'] = 0.05
xgb_params_A['n_estimators'] = 800
xgb_params_A['max_depth'] = 3
xgb_params_A['colsample_bytree'] = 0.9


#lgb
lgb_params_A = {}
lgb_params_A['learning_rate'] = 0.05
lgb_params_A['num_iterations'] = 800
lgb_params_A['max_depth'] = 3
lgb_params_A['colsample_bytree'] = 0.6
lgb_params_A['min_child_weight'] = 4

# Country B Model Parameters

In [13]:
#xgb data
#lgb
lgb_params_B = {}
lgb_params_B['learning_rate'] = 0.01
lgb_params_B['max_depth'] = 8
lgb_params_B['num_iterations'] = 1000
lgb_params_B['colsample_bytree'] = 0.4
lgb_params_B['subsample'] = 0.7

#xgb
xgb_params_B = {}
xgb_params_B['learning_rate'] = 0.05
xgb_params_B['n_estimators'] = 500
xgb_params_B['min_child_weight'] = 2
xgb_params_B['colsample_bytree'] = 0.8
xgb_params_B['max_delta_step'] = 3


# Country C Model Parameters

In [14]:
#xgb data
#xgb
xgb_params_C = {}
xgb_params_C['learning_rate'] = 0.05
xgb_params_C['n_estimators'] = 300
xgb_params_C['max_depth'] = 3
xgb_params_C['colsample_bytree'] = 0.6
xgb_params_C['max_delta_step'] = 3

#lgb
lgb_params_C = {}
lgb_params_C['learning_rate'] = 0.05
lgb_params_C['num_iterations'] = 150
lgb_params_C['max_depth'] = 4

# Parameters tunning

In [15]:
params = {}
params['learning_rate'] = [0.05]
params['n_estimators'] = [800]
params['max_depth'] = [3]
params['colsample_bytree'] = [0.9]
params['min_child_weight'] = [1,2,3,4]

In [20]:
base_model_evaluate(XGBClassifier(**xgb_params_A), x_train_new_A, y_train_A)

(0.2723665910561202, 0.9535956276114173)

In [21]:
base_model_evaluate(XGBClassifier(**xgb_params_A), x_train_new_A, y_train_A, early_stop=True)

[0]	validation_0-logloss:0.676098	validation_1-logloss:0.675327
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 20 rounds.
[1]	validation_0-logloss:0.660681	validation_1-logloss:0.659197
[2]	validation_0-logloss:0.646678	validation_1-logloss:0.644628
[3]	validation_0-logloss:0.633762	validation_1-logloss:0.631091
[4]	validation_0-logloss:0.621899	validation_1-logloss:0.619099
[5]	validation_0-logloss:0.610706	validation_1-logloss:0.607419
[6]	validation_0-logloss:0.600104	validation_1-logloss:0.596614
[7]	validation_0-logloss:0.590095	validation_1-logloss:0.586568
[8]	validation_0-logloss:0.580567	validation_1-logloss:0.576947
[9]	validation_0-logloss:0.571637	validation_1-logloss:0.567968
[10]	validation_0-logloss:0.56328	validation_1-logloss:0.559585
[11]	validation_0-logloss:0.555132	validation_1-logloss:0.552501
[12]	validation_0-logloss:0.547806	validation_1-logloss:0.544983
[

[124]	validation_0-logloss:0.314485	validation_1-logloss:0.329859
[125]	validation_0-logloss:0.313767	validation_1-logloss:0.329377
[126]	validation_0-logloss:0.313035	validation_1-logloss:0.328813
[127]	validation_0-logloss:0.312357	validation_1-logloss:0.328514
[128]	validation_0-logloss:0.311659	validation_1-logloss:0.327878
[129]	validation_0-logloss:0.311028	validation_1-logloss:0.327381
[130]	validation_0-logloss:0.31023	validation_1-logloss:0.326683
[131]	validation_0-logloss:0.309562	validation_1-logloss:0.326339
[132]	validation_0-logloss:0.308824	validation_1-logloss:0.326054
[133]	validation_0-logloss:0.308279	validation_1-logloss:0.325749
[134]	validation_0-logloss:0.307755	validation_1-logloss:0.325474
[135]	validation_0-logloss:0.307088	validation_1-logloss:0.325095
[136]	validation_0-logloss:0.306471	validation_1-logloss:0.324584
[137]	validation_0-logloss:0.305745	validation_1-logloss:0.323972
[138]	validation_0-logloss:0.305168	validation_1-logloss:0.323649
[139]	valid

[249]	validation_0-logloss:0.258352	validation_1-logloss:0.293502
[250]	validation_0-logloss:0.258087	validation_1-logloss:0.293399
[251]	validation_0-logloss:0.257817	validation_1-logloss:0.29318
[252]	validation_0-logloss:0.257558	validation_1-logloss:0.293047
[253]	validation_0-logloss:0.257296	validation_1-logloss:0.292849
[254]	validation_0-logloss:0.257013	validation_1-logloss:0.292663
[255]	validation_0-logloss:0.256735	validation_1-logloss:0.292496
[256]	validation_0-logloss:0.256406	validation_1-logloss:0.292215
[257]	validation_0-logloss:0.256133	validation_1-logloss:0.292014
[258]	validation_0-logloss:0.255868	validation_1-logloss:0.291819
[259]	validation_0-logloss:0.255604	validation_1-logloss:0.291788
[260]	validation_0-logloss:0.255323	validation_1-logloss:0.291674
[261]	validation_0-logloss:0.254989	validation_1-logloss:0.291483
[262]	validation_0-logloss:0.254624	validation_1-logloss:0.291272
[263]	validation_0-logloss:0.254366	validation_1-logloss:0.291196
[264]	valid

[374]	validation_0-logloss:0.23039	validation_1-logloss:0.278845
[375]	validation_0-logloss:0.230239	validation_1-logloss:0.278796
[376]	validation_0-logloss:0.230057	validation_1-logloss:0.27861
[377]	validation_0-logloss:0.229811	validation_1-logloss:0.278584
[378]	validation_0-logloss:0.229708	validation_1-logloss:0.278517
[379]	validation_0-logloss:0.229571	validation_1-logloss:0.278481
[380]	validation_0-logloss:0.22941	validation_1-logloss:0.278549
[381]	validation_0-logloss:0.229243	validation_1-logloss:0.278351
[382]	validation_0-logloss:0.229051	validation_1-logloss:0.278372
[383]	validation_0-logloss:0.228922	validation_1-logloss:0.278296
[384]	validation_0-logloss:0.228717	validation_1-logloss:0.278299
[385]	validation_0-logloss:0.228555	validation_1-logloss:0.278314
[386]	validation_0-logloss:0.228406	validation_1-logloss:0.278191
[387]	validation_0-logloss:0.228257	validation_1-logloss:0.278037
[388]	validation_0-logloss:0.228104	validation_1-logloss:0.277999
[389]	validat

[499]	validation_0-logloss:0.212626	validation_1-logloss:0.273093
[500]	validation_0-logloss:0.212505	validation_1-logloss:0.273044
[501]	validation_0-logloss:0.212379	validation_1-logloss:0.272887
[502]	validation_0-logloss:0.212223	validation_1-logloss:0.272823
[503]	validation_0-logloss:0.212103	validation_1-logloss:0.272865
[504]	validation_0-logloss:0.211974	validation_1-logloss:0.272835
[505]	validation_0-logloss:0.211825	validation_1-logloss:0.272864
[506]	validation_0-logloss:0.211687	validation_1-logloss:0.272736
[507]	validation_0-logloss:0.211582	validation_1-logloss:0.272679
[508]	validation_0-logloss:0.211444	validation_1-logloss:0.272654
[509]	validation_0-logloss:0.211361	validation_1-logloss:0.27272
[510]	validation_0-logloss:0.211223	validation_1-logloss:0.272676
[511]	validation_0-logloss:0.211103	validation_1-logloss:0.2727
[512]	validation_0-logloss:0.210884	validation_1-logloss:0.272792
[513]	validation_0-logloss:0.210792	validation_1-logloss:0.2727
[514]	validatio

...

[279]	validation_0-logloss:0.246485	validation_1-logloss:0.299483
[280]	validation_0-logloss:0.246231	validation_1-logloss:0.299328
[281]	validation_0-logloss:0.245978	validation_1-logloss:0.299341
[282]	validation_0-logloss:0.245723	validation_1-logloss:0.299136
[283]	validation_0-logloss:0.245428	validation_1-logloss:0.299159
[284]	validation_0-logloss:0.245216	validation_1-logloss:0.298952
[285]	validation_0-logloss:0.24501	validation_1-logloss:0.298865
[286]	validation_0-logloss:0.244787	validation_1-logloss:0.298784
[287]	validation_0-logloss:0.244482	validation_1-logloss:0.29874
[288]	validation_0-logloss:0.244283	validation_1-logloss:0.298659
[289]	validation_0-logloss:0.244034	validation_1-logloss:0.298572
[290]	validation_0-logloss:0.243817	validation_1-logloss:0.298252
[291]	validation_0-logloss:0.243577	validation_1-logloss:0.298096
[292]	validation_0-logloss:0.243307	validation_1-logloss:0.298031
[293]	validation_0-logloss:0.243049	validation_1-logloss:0.297852
[294]	valida

[404]	validation_0-logloss:0.220878	validation_1-logloss:0.287951
[405]	validation_0-logloss:0.2207	validation_1-logloss:0.287813
[406]	validation_0-logloss:0.220564	validation_1-logloss:0.287815
[407]	validation_0-logloss:0.220419	validation_1-logloss:0.287839
[408]	validation_0-logloss:0.220259	validation_1-logloss:0.287797
[409]	validation_0-logloss:0.22011	validation_1-logloss:0.287847
[410]	validation_0-logloss:0.219918	validation_1-logloss:0.287823
[411]	validation_0-logloss:0.219765	validation_1-logloss:0.287849
[412]	validation_0-logloss:0.219642	validation_1-logloss:0.287904
[413]	validation_0-logloss:0.219419	validation_1-logloss:0.287848
[414]	validation_0-logloss:0.219293	validation_1-logloss:0.287733
[415]	validation_0-logloss:0.219133	validation_1-logloss:0.287714
[416]	validation_0-logloss:0.218988	validation_1-logloss:0.287631
[417]	validation_0-logloss:0.218834	validation_1-logloss:0.287605
[418]	validation_0-logloss:0.218677	validation_1-logloss:0.287511
[419]	validat

[529]	validation_0-logloss:0.203283	validation_1-logloss:0.284155
[530]	validation_0-logloss:0.203187	validation_1-logloss:0.284169
[531]	validation_0-logloss:0.203088	validation_1-logloss:0.284196
[532]	validation_0-logloss:0.20296	validation_1-logloss:0.284124
[533]	validation_0-logloss:0.20284	validation_1-logloss:0.284168
[534]	validation_0-logloss:0.202751	validation_1-logloss:0.28415
[535]	validation_0-logloss:0.20264	validation_1-logloss:0.284202
[536]	validation_0-logloss:0.202544	validation_1-logloss:0.284245
[537]	validation_0-logloss:0.20236	validation_1-logloss:0.284216
[538]	validation_0-logloss:0.202241	validation_1-logloss:0.284187
[539]	validation_0-logloss:0.202128	validation_1-logloss:0.284089
[540]	validation_0-logloss:0.202028	validation_1-logloss:0.284095
[541]	validation_0-logloss:0.201919	validation_1-logloss:0.284003
[542]	validation_0-logloss:0.201826	validation_1-logloss:0.28399
[543]	validation_0-logloss:0.201704	validation_1-logloss:0.283945
[544]	validation

[654]	validation_0-logloss:0.190188	validation_1-logloss:0.282898
[655]	validation_0-logloss:0.19011	validation_1-logloss:0.282939
[656]	validation_0-logloss:0.190006	validation_1-logloss:0.28292
[657]	validation_0-logloss:0.189902	validation_1-logloss:0.282885
[658]	validation_0-logloss:0.189785	validation_1-logloss:0.282921
[659]	validation_0-logloss:0.189705	validation_1-logloss:0.282932
[660]	validation_0-logloss:0.189581	validation_1-logloss:0.283058
[661]	validation_0-logloss:0.189465	validation_1-logloss:0.282963
[662]	validation_0-logloss:0.189361	validation_1-logloss:0.283083
[663]	validation_0-logloss:0.189275	validation_1-logloss:0.283101
[664]	validation_0-logloss:0.189185	validation_1-logloss:0.283161
[665]	validation_0-logloss:0.189116	validation_1-logloss:0.283163
[666]	validation_0-logloss:0.189027	validation_1-logloss:0.283148
[667]	validation_0-logloss:0.188876	validation_1-logloss:0.283139
[668]	validation_0-logloss:0.188811	validation_1-logloss:0.283139
[669]	valida

(0.27263859509054705, 0.9535879146352831)

# Model

In [22]:
clf1_A = LGBMClassifier(**lgb_params_A)
clf2_A = XGBClassifier(**xgb_params_A)
clf3_A = CatBoostClassifier(loss_function='Logloss')
clf4_A = LogisticRegression()

clf1_B = LGBMClassifier(**lgb_params_B)
clf2_B = XGBClassifier(**xgb_params_B)
clf3_B = CatBoostClassifier(loss_function='Logloss')
clf4_B = LogisticRegression()

clf1_C = LGBMClassifier(**lgb_params_C)
clf2_C = XGBClassifier(**xgb_params_C)
clf3_C = CatBoostClassifier(loss_function='Logloss')
clf4_C = LogisticRegression()

model_A = VotingClassifier(estimators=[('lgb', clf1_A), ('xgb', clf2_A), ('cat', clf3_A), ('lr', clf4_A)], voting='soft')
model_B = VotingClassifier(estimators=[('lgb', clf1_B), ('xgb', clf2_B), ('cat', clf3_B), ('lr', clf4_B)], voting='soft')
model_C = VotingClassifier(estimators=[('lgb', clf1_C), ('xgb', clf2_C), ('cat', clf3_C), ('lr', clf4_C)], voting='soft')

model_A.fit(x_train_new_A, y_train_A)
model_B.fit(x_train_new_B, y_train_B)
model_C.fit(x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6744178	total: 103ms	remaining: 1m 43s
1:	learn: 0.6570813	total: 144ms	remaining: 1m 12s
2:	learn: 0.6398346	total: 179ms	remaining: 59.4s
3:	learn: 0.6246939	total: 209ms	remaining: 52.1s
4:	learn: 0.6096077	total: 245ms	remaining: 48.8s
5:	learn: 0.5966743	total: 327ms	remaining: 54.1s
6:	learn: 0.5846141	total: 432ms	remaining: 1m 1s
7:	learn: 0.5737787	total: 514ms	remaining: 1m 3s
8:	learn: 0.5617385	total: 551ms	remaining: 1m
9:	learn: 0.5532990	total: 584ms	remaining: 57.9s
10:	learn: 0.5446912	total: 621ms	remaining: 55.9s
11:	learn: 0.5355027	total: 655ms	remaining: 54s
12:	learn: 0.5263544	total: 689ms	remaining: 52.3s
13:	learn: 0.5185460	total: 726ms	remaining: 51.2s
14:	learn: 0.5103192	total: 766ms	remaining: 50.3s
15:	learn: 0.5032630	total: 798ms	remaining: 49.1s
16:	learn: 0.4960306	total: 840ms	remaining: 48.6s
17:	learn: 0.4887758	total: 872ms	remaining: 47.6s
18:	learn: 0.4825301	total: 902ms	remaining: 46.6s
19:	learn: 0.4772474	total: 936ms	remaining:

161:	learn: 0.2872582	total: 6.35s	remaining: 32.9s
162:	learn: 0.2868472	total: 6.39s	remaining: 32.8s
163:	learn: 0.2864404	total: 6.43s	remaining: 32.8s
164:	learn: 0.2862641	total: 6.46s	remaining: 32.7s
165:	learn: 0.2859509	total: 6.49s	remaining: 32.6s
166:	learn: 0.2854139	total: 6.53s	remaining: 32.6s
167:	learn: 0.2850521	total: 6.56s	remaining: 32.5s
168:	learn: 0.2847708	total: 6.6s	remaining: 32.4s
169:	learn: 0.2842346	total: 6.63s	remaining: 32.4s
170:	learn: 0.2838254	total: 6.67s	remaining: 32.3s
171:	learn: 0.2835321	total: 6.7s	remaining: 32.3s
172:	learn: 0.2830608	total: 6.74s	remaining: 32.2s
173:	learn: 0.2827039	total: 6.77s	remaining: 32.2s
174:	learn: 0.2822863	total: 6.81s	remaining: 32.1s
175:	learn: 0.2818895	total: 6.84s	remaining: 32s
176:	learn: 0.2815174	total: 6.87s	remaining: 32s
177:	learn: 0.2811894	total: 6.91s	remaining: 31.9s
178:	learn: 0.2809163	total: 6.95s	remaining: 31.9s
179:	learn: 0.2805320	total: 6.99s	remaining: 31.8s
180:	learn: 0.2800

321:	learn: 0.2424177	total: 12.4s	remaining: 26s
322:	learn: 0.2421381	total: 12.4s	remaining: 26s
323:	learn: 0.2420203	total: 12.4s	remaining: 26s
324:	learn: 0.2417629	total: 12.5s	remaining: 25.9s
325:	learn: 0.2415672	total: 12.5s	remaining: 25.9s
326:	learn: 0.2413109	total: 12.6s	remaining: 25.8s
327:	learn: 0.2410241	total: 12.6s	remaining: 25.8s
328:	learn: 0.2408284	total: 12.6s	remaining: 25.8s
329:	learn: 0.2406282	total: 12.7s	remaining: 25.8s
330:	learn: 0.2404269	total: 12.7s	remaining: 25.7s
331:	learn: 0.2402606	total: 12.8s	remaining: 25.7s
332:	learn: 0.2400311	total: 12.8s	remaining: 25.6s
333:	learn: 0.2398752	total: 12.8s	remaining: 25.6s
334:	learn: 0.2396769	total: 12.9s	remaining: 25.5s
335:	learn: 0.2394451	total: 12.9s	remaining: 25.5s
336:	learn: 0.2392143	total: 12.9s	remaining: 25.4s
337:	learn: 0.2390227	total: 13s	remaining: 25.4s
338:	learn: 0.2388171	total: 13s	remaining: 25.4s
339:	learn: 0.2384700	total: 13s	remaining: 25.3s
340:	learn: 0.2382635	to

484:	learn: 0.2130195	total: 18.2s	remaining: 19.3s
485:	learn: 0.2128376	total: 18.2s	remaining: 19.2s
486:	learn: 0.2127268	total: 18.2s	remaining: 19.2s
487:	learn: 0.2125916	total: 18.3s	remaining: 19.2s
488:	learn: 0.2124518	total: 18.3s	remaining: 19.1s
489:	learn: 0.2123316	total: 18.3s	remaining: 19.1s
490:	learn: 0.2122114	total: 18.4s	remaining: 19.1s
491:	learn: 0.2121025	total: 18.4s	remaining: 19s
492:	learn: 0.2119236	total: 18.4s	remaining: 19s
493:	learn: 0.2117358	total: 18.5s	remaining: 18.9s
494:	learn: 0.2116231	total: 18.5s	remaining: 18.9s
495:	learn: 0.2115123	total: 18.5s	remaining: 18.8s
496:	learn: 0.2114039	total: 18.6s	remaining: 18.8s
497:	learn: 0.2113017	total: 18.6s	remaining: 18.8s
498:	learn: 0.2111418	total: 18.7s	remaining: 18.7s
499:	learn: 0.2110354	total: 18.7s	remaining: 18.7s
500:	learn: 0.2108800	total: 18.7s	remaining: 18.7s
501:	learn: 0.2106707	total: 18.8s	remaining: 18.6s
502:	learn: 0.2104621	total: 18.8s	remaining: 18.6s
503:	learn: 0.21

...

493:	learn: 0.0027250	total: 14.3s	remaining: 14.7s
494:	learn: 0.0027125	total: 14.3s	remaining: 14.6s
495:	learn: 0.0027105	total: 14.4s	remaining: 14.6s
496:	learn: 0.0027060	total: 14.4s	remaining: 14.6s
497:	learn: 0.0027001	total: 14.4s	remaining: 14.5s
498:	learn: 0.0026912	total: 14.4s	remaining: 14.5s
499:	learn: 0.0026828	total: 14.5s	remaining: 14.5s
500:	learn: 0.0026724	total: 14.5s	remaining: 14.4s
501:	learn: 0.0026638	total: 14.5s	remaining: 14.4s
502:	learn: 0.0026568	total: 14.5s	remaining: 14.4s
503:	learn: 0.0026530	total: 14.5s	remaining: 14.3s
504:	learn: 0.0026499	total: 14.6s	remaining: 14.3s
505:	learn: 0.0026431	total: 14.6s	remaining: 14.2s
506:	learn: 0.0026320	total: 14.6s	remaining: 14.2s
507:	learn: 0.0026169	total: 14.6s	remaining: 14.2s
508:	learn: 0.0026042	total: 14.7s	remaining: 14.2s
509:	learn: 0.0025946	total: 14.7s	remaining: 14.1s
510:	learn: 0.0025810	total: 14.7s	remaining: 14.1s
511:	learn: 0.0025662	total: 14.7s	remaining: 14.1s
512:	learn: 

657:	learn: 0.0017781	total: 18.4s	remaining: 9.54s
658:	learn: 0.0017712	total: 18.4s	remaining: 9.51s
659:	learn: 0.0017662	total: 18.4s	remaining: 9.48s
660:	learn: 0.0017603	total: 18.4s	remaining: 9.45s
661:	learn: 0.0017596	total: 18.4s	remaining: 9.42s
662:	learn: 0.0017579	total: 18.5s	remaining: 9.39s
663:	learn: 0.0017539	total: 18.5s	remaining: 9.36s
664:	learn: 0.0017527	total: 18.5s	remaining: 9.33s
665:	learn: 0.0017466	total: 18.5s	remaining: 9.3s
666:	learn: 0.0017339	total: 18.6s	remaining: 9.27s
667:	learn: 0.0017264	total: 18.6s	remaining: 9.24s
668:	learn: 0.0017196	total: 18.6s	remaining: 9.21s
669:	learn: 0.0017124	total: 18.6s	remaining: 9.18s
670:	learn: 0.0017109	total: 18.7s	remaining: 9.16s
671:	learn: 0.0017096	total: 18.7s	remaining: 9.14s
672:	learn: 0.0017036	total: 18.8s	remaining: 9.12s
673:	learn: 0.0016941	total: 18.8s	remaining: 9.1s
674:	learn: 0.0016927	total: 18.8s	remaining: 9.07s
675:	learn: 0.0016916	total: 18.9s	remaining: 9.03s
676:	learn: 0.

819:	learn: 0.0012756	total: 22.4s	remaining: 4.93s
820:	learn: 0.0012698	total: 22.5s	remaining: 4.9s
821:	learn: 0.0012624	total: 22.5s	remaining: 4.88s
822:	learn: 0.0012608	total: 22.5s	remaining: 4.85s
823:	learn: 0.0012598	total: 22.6s	remaining: 4.82s
824:	learn: 0.0012570	total: 22.6s	remaining: 4.79s
825:	learn: 0.0012547	total: 22.6s	remaining: 4.77s
826:	learn: 0.0012528	total: 22.7s	remaining: 4.74s
827:	learn: 0.0012522	total: 22.7s	remaining: 4.71s
828:	learn: 0.0012474	total: 22.7s	remaining: 4.68s
829:	learn: 0.0012440	total: 22.7s	remaining: 4.66s
830:	learn: 0.0012421	total: 22.8s	remaining: 4.63s
831:	learn: 0.0012383	total: 22.8s	remaining: 4.6s
832:	learn: 0.0012364	total: 22.8s	remaining: 4.58s
833:	learn: 0.0012351	total: 22.9s	remaining: 4.55s
834:	learn: 0.0012330	total: 22.9s	remaining: 4.52s
835:	learn: 0.0012302	total: 22.9s	remaining: 4.49s
836:	learn: 0.0012276	total: 22.9s	remaining: 4.46s
837:	learn: 0.0012266	total: 23s	remaining: 4.44s
838:	learn: 0.00

984:	learn: 0.0009613	total: 27.9s	remaining: 425ms
985:	learn: 0.0009567	total: 28s	remaining: 397ms
986:	learn: 0.0009560	total: 28s	remaining: 369ms
987:	learn: 0.0009548	total: 28s	remaining: 340ms
988:	learn: 0.0009544	total: 28s	remaining: 312ms
989:	learn: 0.0009540	total: 28.1s	remaining: 283ms
990:	learn: 0.0009533	total: 28.1s	remaining: 255ms
991:	learn: 0.0009521	total: 28.1s	remaining: 227ms
992:	learn: 0.0009502	total: 28.1s	remaining: 198ms
993:	learn: 0.0009492	total: 28.2s	remaining: 170ms
994:	learn: 0.0009469	total: 28.2s	remaining: 142ms
995:	learn: 0.0009460	total: 28.2s	remaining: 113ms
996:	learn: 0.0009454	total: 28.3s	remaining: 85ms
997:	learn: 0.0009446	total: 28.3s	remaining: 56.7ms
998:	learn: 0.0009435	total: 28.3s	remaining: 28.4ms
999:	learn: 0.0009409	total: 28.3s	remaining: 0us


VotingClassifier(estimators=[('lgb', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.05, max_depth=4, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_iterations=150, num_leaves=31, objective=None,...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [23]:
base_model_evaluate(model_A, x_train_new_A, y_train_A)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6754967	total: 34.3ms	remaining: 34.2s
1:	learn: 0.6595292	total: 62.7ms	remaining: 31.3s
2:	learn: 0.6440598	total: 107ms	remaining: 35.5s
3:	learn: 0.6292225	total: 134ms	remaining: 33.5s
4:	learn: 0.6158349	total: 165ms	remaining: 32.8s
5:	learn: 0.6017876	total: 321ms	remaining: 53.2s
6:	learn: 0.5898218	total: 385ms	remaining: 54.6s
7:	learn: 0.5778029	total: 429ms	remaining: 53.2s
8:	learn: 0.5662871	total: 456ms	remaining: 50.2s
9:	learn: 0.5549819	total: 483ms	remaining: 47.8s
10:	learn: 0.5445988	total: 508ms	remaining: 45.6s
11:	learn: 0.5353030	total: 549ms	remaining: 45.2s
12:	learn: 0.5266176	total: 595ms	remaining: 45.2s
13:	learn: 0.5200888	total: 620ms	remaining: 43.6s
14:	learn: 0.5124226	total: 644ms	remaining: 42.3s
15:	learn: 0.5062222	total: 670ms	remaining: 41.2s
16:	learn: 0.4994280	total: 696ms	remaining: 40.2s
17:	learn: 0.4924911	total: 723ms	remaining: 39.4s
18:	learn: 0.4864228	total: 747ms	remaining: 38.6s
19:	learn: 0.4808621	total: 795ms	remai

164:	learn: 0.2866406	total: 6.38s	remaining: 32.3s
165:	learn: 0.2860512	total: 6.44s	remaining: 32.4s
166:	learn: 0.2855791	total: 6.49s	remaining: 32.4s
167:	learn: 0.2851904	total: 6.53s	remaining: 32.4s
168:	learn: 0.2848241	total: 6.56s	remaining: 32.3s
169:	learn: 0.2842881	total: 6.59s	remaining: 32.2s
170:	learn: 0.2837767	total: 6.63s	remaining: 32.1s
171:	learn: 0.2832713	total: 6.66s	remaining: 32s
172:	learn: 0.2829659	total: 6.68s	remaining: 31.9s
173:	learn: 0.2825584	total: 6.71s	remaining: 31.8s
174:	learn: 0.2821722	total: 6.73s	remaining: 31.7s
175:	learn: 0.2816927	total: 6.76s	remaining: 31.6s
176:	learn: 0.2813929	total: 6.79s	remaining: 31.6s
177:	learn: 0.2811415	total: 6.83s	remaining: 31.5s
178:	learn: 0.2806790	total: 6.86s	remaining: 31.5s
179:	learn: 0.2803971	total: 6.88s	remaining: 31.4s
180:	learn: 0.2800152	total: 6.91s	remaining: 31.3s
181:	learn: 0.2795464	total: 6.94s	remaining: 31.2s
182:	learn: 0.2791690	total: 6.96s	remaining: 31.1s
183:	learn: 0.

323:	learn: 0.2375782	total: 12.4s	remaining: 26s
324:	learn: 0.2373095	total: 12.5s	remaining: 26s
325:	learn: 0.2371623	total: 12.6s	remaining: 26s
326:	learn: 0.2368684	total: 12.6s	remaining: 25.9s
327:	learn: 0.2366896	total: 12.6s	remaining: 25.9s
328:	learn: 0.2364725	total: 12.7s	remaining: 25.9s
329:	learn: 0.2363299	total: 12.7s	remaining: 25.9s
330:	learn: 0.2360905	total: 12.8s	remaining: 25.8s
331:	learn: 0.2358996	total: 12.8s	remaining: 25.8s
332:	learn: 0.2357239	total: 12.8s	remaining: 25.7s
333:	learn: 0.2355337	total: 12.9s	remaining: 25.6s
334:	learn: 0.2353433	total: 12.9s	remaining: 25.6s
335:	learn: 0.2350112	total: 12.9s	remaining: 25.5s
336:	learn: 0.2347529	total: 13s	remaining: 25.5s
337:	learn: 0.2345445	total: 13s	remaining: 25.4s
338:	learn: 0.2344015	total: 13s	remaining: 25.4s
339:	learn: 0.2341811	total: 13.1s	remaining: 25.4s
340:	learn: 0.2338828	total: 13.1s	remaining: 25.3s
341:	learn: 0.2336140	total: 13.1s	remaining: 25.3s
342:	learn: 0.2333043	to

486:	learn: 0.2053628	total: 18.3s	remaining: 19.3s
487:	learn: 0.2051996	total: 18.3s	remaining: 19.2s
488:	learn: 0.2049334	total: 18.3s	remaining: 19.2s
489:	learn: 0.2047905	total: 18.4s	remaining: 19.1s
490:	learn: 0.2045836	total: 18.4s	remaining: 19.1s
491:	learn: 0.2043713	total: 18.4s	remaining: 19s
492:	learn: 0.2041871	total: 18.5s	remaining: 19s
493:	learn: 0.2040327	total: 18.5s	remaining: 18.9s
494:	learn: 0.2039121	total: 18.5s	remaining: 18.9s
495:	learn: 0.2037323	total: 18.6s	remaining: 18.9s
496:	learn: 0.2035815	total: 18.6s	remaining: 18.8s
497:	learn: 0.2034386	total: 18.6s	remaining: 18.8s
498:	learn: 0.2033072	total: 18.6s	remaining: 18.7s
499:	learn: 0.2030872	total: 18.7s	remaining: 18.7s
500:	learn: 0.2029571	total: 18.7s	remaining: 18.6s
501:	learn: 0.2028317	total: 18.7s	remaining: 18.6s
502:	learn: 0.2027132	total: 18.8s	remaining: 18.5s
503:	learn: 0.2025671	total: 18.8s	remaining: 18.5s
504:	learn: 0.2024206	total: 18.8s	remaining: 18.5s
505:	learn: 0.20

...

492:	learn: 0.1998560	total: 14.3s	remaining: 14.7s
493:	learn: 0.1997228	total: 14.3s	remaining: 14.7s
494:	learn: 0.1995294	total: 14.3s	remaining: 14.6s
495:	learn: 0.1993667	total: 14.4s	remaining: 14.6s
496:	learn: 0.1991809	total: 14.4s	remaining: 14.6s
497:	learn: 0.1990517	total: 14.4s	remaining: 14.5s
498:	learn: 0.1989664	total: 14.4s	remaining: 14.5s
499:	learn: 0.1988068	total: 14.5s	remaining: 14.5s
500:	learn: 0.1986017	total: 14.5s	remaining: 14.5s
501:	learn: 0.1984352	total: 14.6s	remaining: 14.5s
502:	learn: 0.1981933	total: 14.6s	remaining: 14.5s
503:	learn: 0.1980232	total: 14.7s	remaining: 14.5s
504:	learn: 0.1978592	total: 14.7s	remaining: 14.5s
505:	learn: 0.1976179	total: 14.8s	remaining: 14.5s
506:	learn: 0.1974106	total: 14.9s	remaining: 14.5s
507:	learn: 0.1972183	total: 14.9s	remaining: 14.5s
508:	learn: 0.1970938	total: 15s	remaining: 14.5s
509:	learn: 0.1969072	total: 15.1s	remaining: 14.5s
510:	learn: 0.1968097	total: 15.1s	remaining: 14.5s
511:	learn: 0.

653:	learn: 0.1769559	total: 21.2s	remaining: 11.2s
654:	learn: 0.1768291	total: 21.2s	remaining: 11.2s
655:	learn: 0.1767508	total: 21.3s	remaining: 11.2s
656:	learn: 0.1766091	total: 21.3s	remaining: 11.1s
657:	learn: 0.1764831	total: 21.4s	remaining: 11.1s
658:	learn: 0.1764239	total: 21.5s	remaining: 11.1s
659:	learn: 0.1763322	total: 21.5s	remaining: 11.1s
660:	learn: 0.1761914	total: 21.6s	remaining: 11.1s
661:	learn: 0.1760868	total: 21.6s	remaining: 11s
662:	learn: 0.1759627	total: 21.7s	remaining: 11s
663:	learn: 0.1758394	total: 21.7s	remaining: 11s
664:	learn: 0.1756663	total: 21.8s	remaining: 11s
665:	learn: 0.1754885	total: 21.8s	remaining: 11s
666:	learn: 0.1754167	total: 21.9s	remaining: 10.9s
667:	learn: 0.1753415	total: 21.9s	remaining: 10.9s
668:	learn: 0.1751521	total: 22s	remaining: 10.9s
669:	learn: 0.1750315	total: 22s	remaining: 10.8s
670:	learn: 0.1748697	total: 22.1s	remaining: 10.8s
671:	learn: 0.1747447	total: 22.1s	remaining: 10.8s
672:	learn: 0.1746813	tota

814:	learn: 0.1585598	total: 26.9s	remaining: 6.1s
815:	learn: 0.1584926	total: 27s	remaining: 6.08s
816:	learn: 0.1583226	total: 27s	remaining: 6.05s
817:	learn: 0.1582398	total: 27.1s	remaining: 6.02s
818:	learn: 0.1581566	total: 27.1s	remaining: 6s
819:	learn: 0.1580541	total: 27.2s	remaining: 5.96s
820:	learn: 0.1579771	total: 27.2s	remaining: 5.93s
821:	learn: 0.1578455	total: 27.3s	remaining: 5.9s
822:	learn: 0.1577456	total: 27.3s	remaining: 5.87s
823:	learn: 0.1576681	total: 27.3s	remaining: 5.84s
824:	learn: 0.1575357	total: 27.4s	remaining: 5.8s
825:	learn: 0.1573856	total: 27.4s	remaining: 5.77s
826:	learn: 0.1573406	total: 27.4s	remaining: 5.74s
827:	learn: 0.1572529	total: 27.4s	remaining: 5.7s
828:	learn: 0.1571614	total: 27.5s	remaining: 5.67s
829:	learn: 0.1571004	total: 27.5s	remaining: 5.63s
830:	learn: 0.1569512	total: 27.5s	remaining: 5.6s
831:	learn: 0.1567628	total: 27.6s	remaining: 5.56s
832:	learn: 0.1565595	total: 27.6s	remaining: 5.53s
833:	learn: 0.1564896	to

980:	learn: 0.1434377	total: 32.3s	remaining: 626ms
981:	learn: 0.1434376	total: 32.4s	remaining: 593ms
982:	learn: 0.1433297	total: 32.4s	remaining: 560ms
983:	learn: 0.1432608	total: 32.4s	remaining: 527ms
984:	learn: 0.1432026	total: 32.4s	remaining: 494ms
985:	learn: 0.1431286	total: 32.5s	remaining: 461ms
986:	learn: 0.1430473	total: 32.5s	remaining: 428ms
987:	learn: 0.1430287	total: 32.5s	remaining: 395ms
988:	learn: 0.1429430	total: 32.5s	remaining: 362ms
989:	learn: 0.1429397	total: 32.6s	remaining: 329ms
990:	learn: 0.1428711	total: 32.6s	remaining: 296ms
991:	learn: 0.1428683	total: 32.6s	remaining: 263ms
992:	learn: 0.1427830	total: 32.7s	remaining: 230ms
993:	learn: 0.1427264	total: 32.7s	remaining: 197ms
994:	learn: 0.1426626	total: 32.7s	remaining: 164ms
995:	learn: 0.1425597	total: 32.7s	remaining: 131ms
996:	learn: 0.1423918	total: 32.8s	remaining: 98.6ms
997:	learn: 0.1423585	total: 32.8s	remaining: 65.7ms
998:	learn: 0.1423185	total: 32.8s	remaining: 32.9ms
999:	lear

(0.2682533166357075, 0.9552037871879978)

In [24]:
base_model_evaluate(model_B, x_train_new_B, y_train_B)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6575020	total: 43ms	remaining: 43s
1:	learn: 0.6260536	total: 172ms	remaining: 1m 25s
2:	learn: 0.5948410	total: 237ms	remaining: 1m 18s
3:	learn: 0.5684975	total: 278ms	remaining: 1m 9s
4:	learn: 0.5432007	total: 319ms	remaining: 1m 3s
5:	learn: 0.5171418	total: 343ms	remaining: 56.9s
6:	learn: 0.4947951	total: 380ms	remaining: 53.9s
7:	learn: 0.4754712	total: 405ms	remaining: 50.3s
8:	learn: 0.4575118	total: 445ms	remaining: 49s
9:	learn: 0.4415377	total: 501ms	remaining: 49.6s
10:	learn: 0.4251491	total: 542ms	remaining: 48.7s
11:	learn: 0.4111726	total: 599ms	remaining: 49.3s
12:	learn: 0.3983565	total: 661ms	remaining: 50.2s
13:	learn: 0.3867717	total: 716ms	remaining: 50.4s
14:	learn: 0.3750995	total: 826ms	remaining: 54.3s
15:	learn: 0.3638603	total: 871ms	remaining: 53.5s
16:	learn: 0.3538479	total: 906ms	remaining: 52.4s
17:	learn: 0.3447796	total: 965ms	remaining: 52.6s
18:	learn: 0.3360834	total: 1.02s	remaining: 52.9s
19:	learn: 0.3273808	total: 1.08s	remaining:

164:	learn: 0.1392756	total: 5.84s	remaining: 29.6s
165:	learn: 0.1389509	total: 5.86s	remaining: 29.5s
166:	learn: 0.1384744	total: 5.88s	remaining: 29.3s
167:	learn: 0.1381992	total: 5.9s	remaining: 29.2s
168:	learn: 0.1379282	total: 5.92s	remaining: 29.1s
169:	learn: 0.1376619	total: 5.95s	remaining: 29s
170:	learn: 0.1370909	total: 5.97s	remaining: 28.9s
171:	learn: 0.1366662	total: 5.99s	remaining: 28.8s
172:	learn: 0.1363344	total: 6.01s	remaining: 28.7s
173:	learn: 0.1359850	total: 6.03s	remaining: 28.6s
174:	learn: 0.1356233	total: 6.05s	remaining: 28.5s
175:	learn: 0.1351360	total: 6.07s	remaining: 28.4s
176:	learn: 0.1347608	total: 6.09s	remaining: 28.3s
177:	learn: 0.1342987	total: 6.1s	remaining: 28.2s
178:	learn: 0.1338178	total: 6.13s	remaining: 28.1s
179:	learn: 0.1335294	total: 6.14s	remaining: 28s
180:	learn: 0.1330742	total: 6.17s	remaining: 27.9s
181:	learn: 0.1323954	total: 6.19s	remaining: 27.8s
182:	learn: 0.1321182	total: 6.2s	remaining: 27.7s
183:	learn: 0.13162

324:	learn: 0.0928956	total: 9.92s	remaining: 20.6s
325:	learn: 0.0928159	total: 9.95s	remaining: 20.6s
326:	learn: 0.0926034	total: 9.96s	remaining: 20.5s
327:	learn: 0.0924883	total: 9.98s	remaining: 20.4s
328:	learn: 0.0922543	total: 10s	remaining: 20.4s
329:	learn: 0.0920037	total: 10.1s	remaining: 20.4s
330:	learn: 0.0916778	total: 10.1s	remaining: 20.4s
331:	learn: 0.0915008	total: 10.1s	remaining: 20.4s
332:	learn: 0.0912917	total: 10.1s	remaining: 20.3s
333:	learn: 0.0910461	total: 10.2s	remaining: 20.3s
334:	learn: 0.0907061	total: 10.2s	remaining: 20.3s
335:	learn: 0.0905462	total: 10.2s	remaining: 20.3s
336:	learn: 0.0903378	total: 10.3s	remaining: 20.2s
337:	learn: 0.0900258	total: 10.3s	remaining: 20.3s
338:	learn: 0.0898115	total: 10.4s	remaining: 20.3s
339:	learn: 0.0896345	total: 10.4s	remaining: 20.2s
340:	learn: 0.0893706	total: 10.4s	remaining: 20.2s
341:	learn: 0.0892572	total: 10.5s	remaining: 20.2s
342:	learn: 0.0889578	total: 10.5s	remaining: 20.2s
343:	learn: 0.

483:	learn: 0.0633091	total: 14.7s	remaining: 15.7s
484:	learn: 0.0631765	total: 14.8s	remaining: 15.7s
485:	learn: 0.0630273	total: 14.8s	remaining: 15.6s
486:	learn: 0.0628239	total: 14.8s	remaining: 15.6s
487:	learn: 0.0627038	total: 14.8s	remaining: 15.6s
488:	learn: 0.0625569	total: 14.8s	remaining: 15.5s
489:	learn: 0.0623954	total: 14.9s	remaining: 15.5s
490:	learn: 0.0622901	total: 14.9s	remaining: 15.4s
491:	learn: 0.0621429	total: 14.9s	remaining: 15.4s
492:	learn: 0.0620365	total: 15s	remaining: 15.4s
493:	learn: 0.0618775	total: 15s	remaining: 15.3s
494:	learn: 0.0617506	total: 15s	remaining: 15.3s
495:	learn: 0.0616331	total: 15s	remaining: 15.3s
496:	learn: 0.0614896	total: 15s	remaining: 15.2s
497:	learn: 0.0613652	total: 15.1s	remaining: 15.2s
498:	learn: 0.0612077	total: 15.1s	remaining: 15.2s
499:	learn: 0.0610635	total: 15.1s	remaining: 15.1s
500:	learn: 0.0609855	total: 15.2s	remaining: 15.1s
501:	learn: 0.0607711	total: 15.2s	remaining: 15.1s
502:	learn: 0.0605587	

...

492:	learn: 0.0618368	total: 12.7s	remaining: 13.1s
493:	learn: 0.0616794	total: 12.8s	remaining: 13.1s
494:	learn: 0.0615637	total: 12.8s	remaining: 13.1s
495:	learn: 0.0613352	total: 12.8s	remaining: 13s
496:	learn: 0.0611931	total: 12.8s	remaining: 13s
497:	learn: 0.0611255	total: 12.9s	remaining: 13s
498:	learn: 0.0609637	total: 12.9s	remaining: 12.9s
499:	learn: 0.0608397	total: 12.9s	remaining: 12.9s
500:	learn: 0.0605891	total: 12.9s	remaining: 12.9s
501:	learn: 0.0604679	total: 13s	remaining: 12.8s
502:	learn: 0.0603175	total: 13s	remaining: 12.8s
503:	learn: 0.0600451	total: 13s	remaining: 12.8s
504:	learn: 0.0598300	total: 13s	remaining: 12.8s
505:	learn: 0.0597162	total: 13s	remaining: 12.7s
506:	learn: 0.0595291	total: 13.1s	remaining: 12.7s
507:	learn: 0.0593967	total: 13.1s	remaining: 12.7s
508:	learn: 0.0593470	total: 13.1s	remaining: 12.7s
509:	learn: 0.0591432	total: 13.1s	remaining: 12.6s
510:	learn: 0.0590469	total: 13.2s	remaining: 12.6s
511:	learn: 0.0588086	total:

655:	learn: 0.0408706	total: 16.6s	remaining: 8.69s
656:	learn: 0.0407847	total: 16.6s	remaining: 8.67s
657:	learn: 0.0407297	total: 16.6s	remaining: 8.64s
658:	learn: 0.0406045	total: 16.6s	remaining: 8.61s
659:	learn: 0.0405145	total: 16.7s	remaining: 8.58s
660:	learn: 0.0404038	total: 16.7s	remaining: 8.55s
661:	learn: 0.0403157	total: 16.7s	remaining: 8.53s
662:	learn: 0.0402086	total: 16.7s	remaining: 8.51s
663:	learn: 0.0401226	total: 16.8s	remaining: 8.48s
664:	learn: 0.0399512	total: 16.8s	remaining: 8.45s
665:	learn: 0.0399100	total: 16.8s	remaining: 8.42s
666:	learn: 0.0398634	total: 16.8s	remaining: 8.4s
667:	learn: 0.0397512	total: 16.8s	remaining: 8.37s
668:	learn: 0.0396972	total: 16.9s	remaining: 8.34s
669:	learn: 0.0395213	total: 16.9s	remaining: 8.32s
670:	learn: 0.0394121	total: 16.9s	remaining: 8.29s
671:	learn: 0.0393378	total: 16.9s	remaining: 8.27s
672:	learn: 0.0391841	total: 17s	remaining: 8.24s
673:	learn: 0.0390534	total: 17s	remaining: 8.21s
674:	learn: 0.039

820:	learn: 0.0289745	total: 20.2s	remaining: 4.4s
821:	learn: 0.0288992	total: 20.2s	remaining: 4.38s
822:	learn: 0.0288516	total: 20.2s	remaining: 4.35s
823:	learn: 0.0287723	total: 20.3s	remaining: 4.33s
824:	learn: 0.0286784	total: 20.3s	remaining: 4.3s
825:	learn: 0.0286233	total: 20.3s	remaining: 4.28s
826:	learn: 0.0285725	total: 20.3s	remaining: 4.25s
827:	learn: 0.0285128	total: 20.3s	remaining: 4.23s
828:	learn: 0.0284270	total: 20.4s	remaining: 4.2s
829:	learn: 0.0283546	total: 20.4s	remaining: 4.18s
830:	learn: 0.0283070	total: 20.4s	remaining: 4.15s
831:	learn: 0.0282521	total: 20.4s	remaining: 4.13s
832:	learn: 0.0282001	total: 20.5s	remaining: 4.1s
833:	learn: 0.0281605	total: 20.5s	remaining: 4.08s
834:	learn: 0.0281156	total: 20.5s	remaining: 4.05s
835:	learn: 0.0280240	total: 20.5s	remaining: 4.03s
836:	learn: 0.0279876	total: 20.5s	remaining: 4s
837:	learn: 0.0279727	total: 20.6s	remaining: 3.98s
838:	learn: 0.0279172	total: 20.6s	remaining: 3.95s
839:	learn: 0.02784

979:	learn: 0.0214263	total: 24.3s	remaining: 495ms
980:	learn: 0.0213982	total: 24.3s	remaining: 471ms
981:	learn: 0.0213665	total: 24.3s	remaining: 446ms
982:	learn: 0.0213179	total: 24.3s	remaining: 421ms
983:	learn: 0.0212436	total: 24.4s	remaining: 396ms
984:	learn: 0.0212076	total: 24.4s	remaining: 371ms
985:	learn: 0.0211934	total: 24.4s	remaining: 347ms
986:	learn: 0.0211628	total: 24.4s	remaining: 322ms
987:	learn: 0.0211201	total: 24.5s	remaining: 297ms
988:	learn: 0.0210846	total: 24.5s	remaining: 272ms
989:	learn: 0.0210204	total: 24.5s	remaining: 247ms
990:	learn: 0.0209930	total: 24.5s	remaining: 223ms
991:	learn: 0.0209660	total: 24.5s	remaining: 198ms
992:	learn: 0.0209127	total: 24.6s	remaining: 173ms
993:	learn: 0.0208767	total: 24.6s	remaining: 148ms
994:	learn: 0.0208378	total: 24.6s	remaining: 124ms
995:	learn: 0.0207985	total: 24.6s	remaining: 99ms
996:	learn: 0.0207539	total: 24.7s	remaining: 74.2ms
997:	learn: 0.0207163	total: 24.7s	remaining: 49.5ms
998:	learn:

(0.19489071593066715, 0.8829844357443475)

In [25]:
base_model_evaluate(model_C, x_train_new_C, y_train_C)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0:	learn: 0.6242443	total: 33.1ms	remaining: 33.1s
1:	learn: 0.5703115	total: 69.2ms	remaining: 34.5s
2:	learn: 0.5028027	total: 103ms	remaining: 34.3s
3:	learn: 0.4548693	total: 119ms	remaining: 29.6s
4:	learn: 0.4114305	total: 135ms	remaining: 26.9s
5:	learn: 0.3714354	total: 152ms	remaining: 25.2s
6:	learn: 0.3368368	total: 169ms	remaining: 24s
7:	learn: 0.3072734	total: 197ms	remaining: 24.4s
8:	learn: 0.2804300	total: 212ms	remaining: 23.4s
9:	learn: 0.2560611	total: 229ms	remaining: 22.7s
10:	learn: 0.2325763	total: 248ms	remaining: 22.3s
11:	learn: 0.2122562	total: 267ms	remaining: 22s
12:	learn: 0.1950117	total: 294ms	remaining: 22.3s
13:	learn: 0.1808713	total: 317ms	remaining: 22.3s
14:	learn: 0.1668025	total: 343ms	remaining: 22.5s
15:	learn: 0.1537208	total: 359ms	remaining: 22.1s
16:	learn: 0.1428127	total: 385ms	remaining: 22.3s
17:	learn: 0.1332645	total: 401ms	remaining: 21.9s
18:	learn: 0.1243578	total: 417ms	remaining: 21.5s
19:	learn: 0.1160904	total: 443ms	remaining

164:	learn: 0.0089070	total: 3.21s	remaining: 16.3s
165:	learn: 0.0088503	total: 3.24s	remaining: 16.3s
166:	learn: 0.0088051	total: 3.25s	remaining: 16.2s
167:	learn: 0.0087157	total: 3.27s	remaining: 16.2s
168:	learn: 0.0086760	total: 3.29s	remaining: 16.2s
169:	learn: 0.0086286	total: 3.3s	remaining: 16.1s
170:	learn: 0.0085709	total: 3.32s	remaining: 16.1s
171:	learn: 0.0085298	total: 3.35s	remaining: 16.1s
172:	learn: 0.0084665	total: 3.37s	remaining: 16.1s
173:	learn: 0.0084143	total: 3.39s	remaining: 16.1s
174:	learn: 0.0083668	total: 3.41s	remaining: 16.1s
175:	learn: 0.0083325	total: 3.42s	remaining: 16s
176:	learn: 0.0082956	total: 3.45s	remaining: 16s
177:	learn: 0.0082603	total: 3.47s	remaining: 16s
178:	learn: 0.0082071	total: 3.48s	remaining: 16s
179:	learn: 0.0081481	total: 3.5s	remaining: 15.9s
180:	learn: 0.0081244	total: 3.52s	remaining: 15.9s
181:	learn: 0.0080783	total: 3.54s	remaining: 15.9s
182:	learn: 0.0080478	total: 3.55s	remaining: 15.9s
183:	learn: 0.0080206	

326:	learn: 0.0040888	total: 6.17s	remaining: 12.7s
327:	learn: 0.0040772	total: 6.2s	remaining: 12.7s
328:	learn: 0.0040601	total: 6.21s	remaining: 12.7s
329:	learn: 0.0040494	total: 6.23s	remaining: 12.7s
330:	learn: 0.0040246	total: 6.25s	remaining: 12.6s
331:	learn: 0.0040194	total: 6.27s	remaining: 12.6s
332:	learn: 0.0039940	total: 6.28s	remaining: 12.6s
333:	learn: 0.0039746	total: 6.31s	remaining: 12.6s
334:	learn: 0.0039440	total: 6.33s	remaining: 12.6s
335:	learn: 0.0039378	total: 6.35s	remaining: 12.5s
336:	learn: 0.0039150	total: 6.36s	remaining: 12.5s
337:	learn: 0.0039007	total: 6.38s	remaining: 12.5s
338:	learn: 0.0038783	total: 6.41s	remaining: 12.5s
339:	learn: 0.0038683	total: 6.42s	remaining: 12.5s
340:	learn: 0.0038514	total: 6.44s	remaining: 12.4s
341:	learn: 0.0038301	total: 6.46s	remaining: 12.4s
342:	learn: 0.0038188	total: 6.48s	remaining: 12.4s
343:	learn: 0.0037976	total: 6.49s	remaining: 12.4s
344:	learn: 0.0037918	total: 6.51s	remaining: 12.4s
345:	learn: 0

492:	learn: 0.0022220	total: 9.91s	remaining: 10.2s
493:	learn: 0.0022158	total: 10.1s	remaining: 10.3s
494:	learn: 0.0022078	total: 10.1s	remaining: 10.3s
495:	learn: 0.0022035	total: 10.2s	remaining: 10.3s
496:	learn: 0.0021986	total: 10.2s	remaining: 10.3s
497:	learn: 0.0021940	total: 10.2s	remaining: 10.3s
498:	learn: 0.0021906	total: 10.2s	remaining: 10.3s
499:	learn: 0.0021842	total: 10.2s	remaining: 10.2s
500:	learn: 0.0021801	total: 10.3s	remaining: 10.2s
501:	learn: 0.0021751	total: 10.3s	remaining: 10.2s
502:	learn: 0.0021670	total: 10.3s	remaining: 10.2s
503:	learn: 0.0021651	total: 10.3s	remaining: 10.1s
504:	learn: 0.0021603	total: 10.3s	remaining: 10.1s
505:	learn: 0.0021572	total: 10.3s	remaining: 10.1s
506:	learn: 0.0021542	total: 10.4s	remaining: 10.1s
507:	learn: 0.0021472	total: 10.4s	remaining: 10.1s
508:	learn: 0.0021398	total: 10.4s	remaining: 10s
509:	learn: 0.0021355	total: 10.4s	remaining: 10s
510:	learn: 0.0021309	total: 10.4s	remaining: 9.98s
511:	learn: 0.00

...

662:	learn: 0.0013775	total: 14.8s	remaining: 7.53s
663:	learn: 0.0013691	total: 14.9s	remaining: 7.52s
664:	learn: 0.0013662	total: 14.9s	remaining: 7.49s
665:	learn: 0.0013627	total: 14.9s	remaining: 7.46s
666:	learn: 0.0013565	total: 14.9s	remaining: 7.44s
667:	learn: 0.0013553	total: 14.9s	remaining: 7.42s
668:	learn: 0.0013538	total: 15s	remaining: 7.4s
669:	learn: 0.0013527	total: 15s	remaining: 7.37s
670:	learn: 0.0013511	total: 15s	remaining: 7.35s
671:	learn: 0.0013497	total: 15s	remaining: 7.32s
672:	learn: 0.0013479	total: 15s	remaining: 7.3s
673:	learn: 0.0013472	total: 15s	remaining: 7.28s
674:	learn: 0.0013465	total: 15.1s	remaining: 7.25s
675:	learn: 0.0013451	total: 15.1s	remaining: 7.23s
676:	learn: 0.0013428	total: 15.1s	remaining: 7.2s
677:	learn: 0.0013404	total: 15.1s	remaining: 7.18s
678:	learn: 0.0013357	total: 15.1s	remaining: 7.16s
679:	learn: 0.0013328	total: 15.2s	remaining: 7.13s
680:	learn: 0.0013298	total: 15.2s	remaining: 7.11s
681:	learn: 0.0013268	total

827:	learn: 0.0009995	total: 18s	remaining: 3.73s
828:	learn: 0.0009991	total: 18s	remaining: 3.71s
829:	learn: 0.0009986	total: 18s	remaining: 3.69s
830:	learn: 0.0009973	total: 18s	remaining: 3.67s
831:	learn: 0.0009961	total: 18s	remaining: 3.64s
832:	learn: 0.0009953	total: 18.1s	remaining: 3.62s
833:	learn: 0.0009941	total: 18.1s	remaining: 3.6s
834:	learn: 0.0009934	total: 18.1s	remaining: 3.58s
835:	learn: 0.0009913	total: 18.1s	remaining: 3.56s
836:	learn: 0.0009902	total: 18.1s	remaining: 3.53s
837:	learn: 0.0009850	total: 18.2s	remaining: 3.51s
838:	learn: 0.0009833	total: 18.2s	remaining: 3.49s
839:	learn: 0.0009827	total: 18.2s	remaining: 3.47s
840:	learn: 0.0009822	total: 18.2s	remaining: 3.45s
841:	learn: 0.0009817	total: 18.3s	remaining: 3.42s
842:	learn: 0.0009808	total: 18.3s	remaining: 3.4s
843:	learn: 0.0009791	total: 18.3s	remaining: 3.38s
844:	learn: 0.0009785	total: 18.3s	remaining: 3.36s
845:	learn: 0.0009780	total: 18.3s	remaining: 3.33s
846:	learn: 0.0009777	to

987:	learn: 0.0007923	total: 21.1s	remaining: 256ms
988:	learn: 0.0007921	total: 21.1s	remaining: 235ms
989:	learn: 0.0007915	total: 21.1s	remaining: 213ms
990:	learn: 0.0007911	total: 21.1s	remaining: 192ms
991:	learn: 0.0007896	total: 21.2s	remaining: 171ms
992:	learn: 0.0007885	total: 21.2s	remaining: 149ms
993:	learn: 0.0007881	total: 21.2s	remaining: 128ms
994:	learn: 0.0007870	total: 21.2s	remaining: 107ms
995:	learn: 0.0007864	total: 21.2s	remaining: 85.3ms
996:	learn: 0.0007860	total: 21.3s	remaining: 64ms
997:	learn: 0.0007844	total: 21.3s	remaining: 42.6ms
998:	learn: 0.0007840	total: 21.3s	remaining: 21.3ms
999:	learn: 0.0007789	total: 21.3s	remaining: 0us


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


(0.02627794116443121, 0.9993937843101148)

In [26]:
A_pred = model_A.predict_proba(x_test_new_A)
B_pred = model_B.predict_proba(x_test_new_B)
C_pred = model_C.predict_proba(x_test_new_C)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [27]:
A_sub = make_country_sub(A_pred, test_A, 'A')
B_sub = make_country_sub(B_pred, test_B, 'B')
C_sub = make_country_sub(C_pred, test_C, 'C')

In [28]:
submission = pd.concat([A_sub, B_sub, C_sub])

In [29]:
submission.to_csv('0224better.csv')